## SET UP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/LauraMJanssen/srcnn_fsrcnn_srgan.git
%cd srcnn_fsrcnn_srgan

## TRAIN

i. SRCNN

For transfer learning: \
remove '#' in line 31 in 'srcnn_fsrcnn_srgan/SRCNN/solver.py'

In [ ]:
!python3 main.py -m srcnn \
                 --lr 0.0003 \
                 --nEpochs 20

In [ ]:
!zip -r /content/srcnn_model.zip '/content/srcnn_fsrcnn_srgan/model_path.pth'

In [ ]:
from google.colab import files
files.download("/content/srcnn_model.zip")

ii. FSRCNN

For transfer learning: \
remove '#' in line 29 in 'srcnn_fsrcnn_srgan/FSRCNN/solver.py'

In [ ]:
!python3 main.py -m srcnn \
                 --lr 0.0003 \
                 --nEpochs 20

In [ ]:
!zip -r /content/fsrcnn_model.zip '/content/srcnn_fsrcnn_srgan/model_path.pth'

In [ ]:
from google.colab import files
files.download("/content/fsrcnn_model.zip")

iii. SRGAN

In [ ]:
!python3 main.py -m srgan \
                 --lr 0.0003 \
                 --nEpochs 20

In [ ]:
!zip -r /content/mGen.zip \
        '/content/srcnn_fsrcnn_srgan/SRGAN_Generator_model_path.pth'

In [ ]:
!zip -r /content/mDis.zip \
        '/content/srcnn_fsrcnn_srgan/SRGAN_Discriminator_model_path.pth'

In [ ]:
from google.colab import files
files.download("/content/mGen.zip")
files.download("/content/mDis.zip")

## TEST

(1) Define quality metric functions

In [16]:
from skimage.metrics import structural_similarity as ssim

# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
         
    # assume RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    # the MSE between the two images is the sum of the squared difference between the two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores

(2) Execute model on specific image:

i. SRCNN & FSRCNN

In [ ]:
!python3 super_resolve.py \
        --input '/content/drive/MyDrive/IVUSImages/LR_Scale4_Valid/001.png' \
        --output 'test.png' \
        --model '/content/srcnn_fsrcnn_srgan/model_path.pth'

ii. SRGAN

In [ ]:
!python3 super_resolve.py \
        --input '/content/drive/MyDrive/IVUSImages/LR_Scale4_Valid/001.png' \
        --output 'test.png' \
        --model '/content/super-resolution/SRGAN_Generator_model_path.pth'

(3) Print results \
(make sure to adapt ref and degraded parts according to image read above)

In [ ]:
import math
import numpy as np
import cv2

HR = cv2.imread('/content/drive/MyDrive/IVUSImages/HR_Valid/001.png')
LR = cv2.imread('/content/drive/MyDrive/IVUSImages/LR_Scale4_Valid/001.png')
SR = cv2.imread('/content/srcnn_fsrcnn_srgan/test.png')
  
# image quality calculations
scores = []
h, w, _ = LR.shape
LR2 = cv2.resize(LR, (4*w, 4*h))
scores.append(compare_images(LR2, HR))

scores.append(compare_images(SR, HR))
 
# print all scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'
        .format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'
        .format(scores[1][0], scores[1][1], scores[1][2]))